#### Подключаемся к серверy

```bash
ssh 305_koryagin@37.139.32.56 -i ./id_rsa_305_koryagin.txt
```

#### Запускаем spark

```python
/spark2.4/bin/pyspark

from pyspark.shell import sc
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
```

In [4]:
def train_test_split_by_week(df, week_col_name, test_size_weeks):
    """
    Разделение на train и test по неделям
    :param df: исходный датафрейм
    :param week_col_name: название колонки с номерами недели в году
    :param test_size_weeks: число недель для теста
    :return: 2 датасета
    """
    threshold_week = int(data.select(F.max(week_col_name)).collect()[0][0]) - test_size_weeks
    train = df.filter(F.col(week_col_name) < threshold_week)
    test = df.filter(F.col(week_col_name) >= threshold_week)
    return train, test

In [3]:
def transform_for_als(df, user_col_name, item_col_name, rating_col_name):
    """
    Преобразование для ALS
    :param df: исходный датафрейм 
    :param user_col_name: имя колонки пользователей/покупателей
    :param item_col_name: имя колонки с товарами
    :param rating_col_name: имя колонки с рейтингом (сумма, количество продаж)
    :return: преобразованный датафрейм
    """
    return df \
        .select(user_col_name, item_col_name, rating_col_name) \
        .groupBy(user_col_name, item_col_name).sum(rating_col_name) \
        .withColumnRenamed(existing='sum(quantity)', new='quantity')

## Создание DataFrame

```python
# для начала готовим DataFrame
data = spark.read.parquet("input_csv_for_recommend_system/data.parquet")
data.show(n=5, truncate=True)
```

<details>
    <summary> → вывод консоли SPARK</summary>
    
![Title](../images/2021-04-25_125511.jpg)

</details>

```python
# Введем колонку с номером недели
data = data.withColumn(colName='week_of_year', col=F.weekofyear(F.col('sale_date_date')))  

# Разделим набор данных на тренировочную и тестовую выборки
data_train, data_test = train_test_split_by_week(df=data, week_col_name='week_of_year', test_size_weeks=3)
```

## Преобразование данных для ALS

```python
# Преобразуем данные для ALS
train = transform_for_als(data_train, 'contact_id', 'product_id', 'quantity')
test = transform_for_als(data_test, 'contact_id', 'product_id', 'quantity')

test.show(n=5, truncate=True)
```

<details>
    <summary> → вывод консоли SPARK</summary>

```bash
+----------+----------+-------------------+
|contact_id|product_id|           quantity|
+----------+----------+-------------------+
|   2737106|     83462|                3.0|
|   1387649|    124839|               20.0|
|    171180|     76190| 0.4000000059604645|
|    107728|    147802|0.30000000447034836|
|    993804|    158586|               30.0|
+----------+----------+-------------------+
```
</details>

## Обучение модели и ее сохранение ее в файл

```python
# Обучениеё
model = ALS.trainImplicit(ratings=train, rank=10, alpha=0.01, nonnegative=True, seed=42)

# Save model
model.save(sc, "ml_models/myCollaborativeFilter")
```